Have you ever wanted to make your text messages more expressive? Your emojifier app will help you do that. 
Rather than writing:
>"Congratulations on the promotion! Let's get coffee and talk. Love you!"   

The emojifier can automatically turn this into:
>"Congratulations on the promotion! 👍  Let's get coffee and talk. ☕️ Love you! ❤️"

You'll implement a model which inputs a sentence (such as "Let's go see the baseball game tonight!") and finds the most appropriate emoji to be used with this sentence (⚾️).

### Using Word Vectors to Improve Emoji Lookups
* In many emoji interfaces, you need to remember that ❤️  is the "heart" symbol rather than the "love" symbol. 
    * In other words, you'll have to remember to type "heart" to find the desired emoji, and typing "love" won't bring up that symbol.
* You can make a more flexible emoji interface by using word vectors!
* When using word vectors, you'll see that even if your training set explicitly relates only a few words to a particular emoji, your algorithm will be able to generalize and associate additional words in the test set to the same emoji.
    * This works even if those additional words don't even appear in the training set. 
    * This allows you to build an accurate classifier mapping from sentences to emojis, even using a small training set. 

### What you'll build:
1. In this exercise, you'll start with a baseline model (Emojifier-V1) using word embeddings.
2. Then you will build a more sophisticated model (Emojifier-V2) that further incorporates an LSTM. 

By the end of this notebook, you'll be able to:

* Create an embedding layer in Keras with pre-trained word vectors
* Explain the advantages and disadvantages of the GloVe algorithm
* Describe how negative sampling learns word vectors more efficiently than other methods
* Build a sentiment classifier using word embeddings
* Build and train a more sophisticated classifier using an LSTM

🏀 👑

👆 😎

(^^^ Emoji for "skills") 

## Packages

In [36]:
import os

# Only the TensorFlow backend supports string inputs.
os.environ["KERAS_BACKEND"] = "tensorflow"

import pathlib
import numpy as np
from tensorflow import data as tf_data
from tensorflow import keras
import pandas as pd
from emo_utils import *
import tensorflow as tf


## Load Dataset: EMOJISET 

You have a tiny dataset (X, Y) where:
- X contains 127 sentences (strings).
- Y contains an integer label between 0 and 4 corresponding to an emoji for each sentence.

<img src="images/data_set.png" style="width:700px;height:300px;">

Load the dataset using the code below. The dataset is split between training (127 examples) and testing (56 examples).

In [37]:
data = pd.read_csv('data/train_emoji.csv', names=["sentence", "emoji", "remove1", "remove_2"])
data.drop(columns=["remove1", "remove_2"], inplace=True)
data_test = pd.read_csv('data/tesss.csv', names=["sentence", "emoji"])

In [95]:
X_train, y_train = data["sentence"], data["emoji"]
X_test, y_test = data_test["sentence"], data_test["emoji"]

In [96]:
for idx in range(10):
    print(X_train[idx], label_to_emoji(y_train[idx]))

never talk to me again 😞
I am proud of your achievements 😄
It is the worst day in my life 😞
Miss you so much ❤️
food is life 🍴
I love you mum ❤️
Stop saying bullshit 😞
congratulations on your acceptance 😄
The assignment is too long  😞
I want to go play ⚾️


In the below code cell, you will find out the sentence with the maximum number of words, and will store it's length in `maxLen` (*i.e., the number of words in the longest sentence, which will be used further*). Let's break down this code for a better understanding.

- The first point to note here is that `split()` breaks a string into a list of it's words. So, if `x` is a string, then `len(x.split())` returns the number of words in that string. You can read more about `split` [here](https://docs.python.org/3/library/stdtypes.html?highlight=split#str.split).

- The second point to note here is the way in which `max` function has been used. As can be read [here](https://docs.python.org/3/library/functions.html#max), apart from an iterable (*which in your case is `X_train`, a list of strings*), this function also has a `key` argument, that can be used to modify the basis on which the largest element in the iterable is chosen.

In this case, `key` has been chosen as the number of words in a string. So the `max` function will return the string with the largest number of words.

In [43]:
maxLen = len(max(X_train, key=lambda x: len(x.split())).split())
maxLen

10

## Create a Vocabulary index

In [44]:
vectorizer = keras.layers.TextVectorization(max_tokens=40000, output_sequence_length=10)
text_ds = tf_data.Dataset.from_tensor_slices(X_train)
vectorizer.adapt(text_ds)

In [45]:
vectorizer.get_vocabulary()[:5]

['', '[UNK]', 'i', 'you', 'is']

In [46]:
len(vectorizer.get_vocabulary())


262

In [47]:
output = vectorizer([["the cat sat on the mat"]])
output.numpy()[0, :6]

array([ 5,  1,  1, 39,  5,  1], dtype=int64)

In [48]:
voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

In [49]:
test = ["i", "love", "to",  "eat", "chinese", "food"]
[word_index[w] for w in test]

[2, 21, 10, 209, 227, 27]

#### One-hot Encoding
* To get your labels into a format suitable for training a softmax classifier, convert $Y$ from its current shape  $(m, 1)$ into a "one-hot representation" $(m, 5)$, 

In [50]:
train_labels = keras.utils.to_categorical(y_train)
test_labels = keras.utils.to_categorical(y_test)

In [51]:
idx = 50
print(f"Sentence '{X_train[idx]}' has label index {y_train[idx]}, which is emoji {label_to_emoji(y_train[idx])}", )
print(f"Label index {y_train[idx]} in one-hot encoding format is {train_labels[idx]}")

Sentence 'I missed you' has label index 0, which is emoji ❤️
Label index 0 in one-hot encoding format is [1. 0. 0. 0. 0.]


## Load pre-trained word embeddings

In [52]:
path_to_glove_file = "data/glove.6B.50d.txt"

embeddings_index = {}
with open(path_to_glove_file) as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print(embeddings_index["cat"])
print("Found %s word vectors." % len(embeddings_index))

[ 0.45281  -0.50108  -0.53714  -0.015697  0.22191   0.54602  -0.67301
 -0.6891    0.63493  -0.19726   0.33685   0.7735    0.90094   0.38488
  0.38367   0.2657   -0.08057   0.61089  -1.2894   -0.22313  -0.61578
  0.21697   0.35614   0.44499   0.60885  -1.1633   -1.1579    0.36118
  0.10466  -0.78325   1.4352    0.18629  -0.26112   0.83275  -0.23123
  0.32481   0.14485  -0.44552   0.33497  -0.95946  -0.097479  0.48138
 -0.43352   0.69455   0.91043  -0.28173   0.41637  -1.2609    0.71278
  0.23782 ]
Found 400000 word vectors.


 However, most deep learning frameworks require that all sequences in the same mini-batch have the **same length**. 

This is what allows vectorization to work: If you had a 3-word sentence and a 4-word sentence, then the computations needed for them are different (one takes 3 steps of an LSTM, one takes 4 steps) so it's just not possible to do them both at the same time.
    
#### Padding Handles Sequences of Varying Length
* The common solution to handling sequences of **different length** is to use padding.  Specifically:
    * Set a maximum sequence length
    * Pad all sequences to have the same length. 
    
#### Example of Padding:
* Given a maximum sequence length of 20, you could pad every sentence with "0"s so that each input sentence is of length 20. 
* Thus, the sentence "I love you" would be represented as $(e_{I}, e_{love}, e_{you}, \vec{0}, \vec{0}, \ldots, \vec{0})$. 
* In this example, any sentences longer than 20 words would have to be truncated. 
* One way to choose the maximum sequence length is to just pick the length of the longest sentence in the training set. 
  
### The Embedding Layer

In Keras, the embedding matrix is represented as a "layer."

* The embedding matrix maps word indices to embedding vectors.
    * The word indices are positive integers.
    * The embedding vectors are dense vectors of fixed size.
    * A "dense" vector is the opposite of a sparse vector. It means that most of its values are non-zero.  As a counter-example, a one-hot encoded vector is not "dense."
* The embedding matrix can be derived in two ways:
    * Training a model to derive the embeddings from scratch. 
    * Using a pretrained embedding.
    
#### Using and Updating Pre-trained Embeddings
In this section, you'll create an [Embedding()](https://www.tensorflow.org/api_docs/python/tf/keras/layers/Embedding) layer in Keras

* You will initialize the Embedding layer with GloVe 50-dimensional vectors. 
* In the code below, you'll observe how Keras allows you to either train or leave this layer fixed.  
    * Because your training set is quite small, you'll leave the GloVe embeddings fixed instead of updating them.

#### Inputs and Outputs to the Embedding Layer

* The `Embedding()` layer's input is an integer matrix of size **(batch size, max input length)**. 
    * This input corresponds to sentences converted into lists of indices (integers).
    * The largest integer (the highest word index) in the input should be no larger than the vocabulary size.
* The embedding layer outputs an array of shape (batch size, max input length, dimension of word vectors).

* The figure shows the propagation of two example sentences through the embedding layer. 
    * Both examples have been zero-padded to a length of `max_len=5`.
    * The word embeddings are 50 units in length.
    * The final dimension of the representation is  `(2,max_len,50)`. 

<img src="images/embedding1.png" style="width:700px;height:250px;">

In [53]:
num_tokens = len(embeddings_index) + 2
embedding_dim = 50
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

Converted 260 words (2 misses)


In [54]:
embedding_layer = keras.layers.Embedding(
    num_tokens,
    embedding_dim,
    trainable=False,
)
embedding_layer.build((1,))
embedding_layer.set_weights([embedding_matrix])

## Build the model

### 2.1 - Model Overview

Here is the Emojifier-v2 you will implement:

<img src="images/emojifier-v2.png" style="width:700px;height:400px;">

In [105]:
int_sequences_input = keras.Input(shape=(None,), dtype="int32")
embedded_sequences = embedding_layer(int_sequences_input)

# Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
# The returned output should be a batch of sequences.
X = keras.layers.LSTM(128, return_sequences=True)(embedded_sequences)

# Add dropout with a probability of 0.5
X =  keras.layers.Dropout(0.5)(X)

# Propagate X trough another LSTM layer with 128-dimensional hidden state
# The returned output should be a single hidden state, not a batch of sequences.
X =  keras.layers.LSTM(128)(X)
# Add dropout with a probability of 0.5
X =  keras.layers.Dropout(0.5)(X)

# Propagate X through a Dense layer with 5 units
outputs =  keras.layers.Dense(5)(X)

model = keras.Model(int_sequences_input, outputs)
model.summary()

Model: "model_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_23 (InputLayer)       [(None, None)]            0         
                                                                 
 embedding_1 (Embedding)     (None, None, 50)          20000100  
                                                                 
 lstm_17 (LSTM)              (None, None, 128)         91648     
                                                                 
 dropout_16 (Dropout)        (None, None, 128)         0         
                                                                 
 lstm_18 (LSTM)              (None, 128)               131584    
                                                                 
 dropout_17 (Dropout)        (None, 128)               0         
                                                                 
 dense_11 (Dense)            (None, 5)                 645

In [106]:
x_train = vectorizer(np.array([[s] for s in X_train])).numpy()
# x_val = vectorizer(np.array([[s] for s in val_samples])).numpy()

y_train = np.array(train_labels)
# y_val = np.array(val_labels)

In [107]:
vectorizer(["funny lol", "lets play baseball", "food is ready for you"])

<tf.Tensor: shape=(3, 10), dtype=int64, numpy=
array([[ 33,  64,   0,   0,   0,   0,   0,   0,   0,   0],
       [ 43, 140,  34,   0,   0,   0,   0,   0,   0,   0],
       [ 27,   4, 131,  12,   3,   0,   0,   0,   0,   0]], dtype=int64)>

In [108]:
model.compile(loss=keras.losses.CategoricalCrossentropy(from_logits=True), optimizer='adam', metrics=['accuracy'])

model.fit(x_train, y_train, epochs=50, batch_size = 32, shuffle=True)

Epoch 1/50
5/5 [==============================] - 5s 30ms/step - loss: 1.5923 - accuracy: 0.2727
Epoch 2/50
5/5 [==============================] - 0s 13ms/step - loss: 1.5232 - accuracy: 0.3182
Epoch 3/50
5/5 [==============================] - 0s 15ms/step - loss: 1.4710 - accuracy: 0.2955
Epoch 4/50
5/5 [==============================] - 0s 15ms/step - loss: 1.3887 - accuracy: 0.4470
Epoch 5/50
5/5 [==============================] - 0s 14ms/step - loss: 1.3052 - accuracy: 0.4848
Epoch 6/50
5/5 [==============================] - 0s 14ms/step - loss: 1.1948 - accuracy: 0.5985
Epoch 7/50
5/5 [==============================] - 0s 14ms/step - loss: 1.0907 - accuracy: 0.6136
Epoch 8/50
5/5 [==============================] - 0s 13ms/step - loss: 1.0371 - accuracy: 0.6439
Epoch 9/50
5/5 [==============================] - 0s 14ms/step - loss: 1.0132 - accuracy: 0.6439
Epoch 10/50
5/5 [==============================] - 0s 14ms/step - loss: 0.8083 - accuracy: 0.7348
Epoch 11/50
5/5 [============

In [109]:
x_test = vectorizer(np.array([[s] for s in X_test])).numpy()
loss, acc = model.evaluate(x_test, test_labels)
print()
print("Test accuracy = ", acc)


2/2 [==============================] - 1s 8ms/step - loss: 0.9256 - accuracy: 0.6964

Test accuracy =  0.6964285969734192


In [110]:
# This code allows you to see the mislabelled examples
string_input = keras.Input(shape=(1,), dtype="string")
x = vectorizer(string_input)
x = model(x)
# Add a softmax activation
outputs =  keras.layers.Activation('softmax')(x)
end_to_end_model = keras.Model(string_input, outputs)

probabilities = end_to_end_model(
        tf.convert_to_tensor(X_test)
)
for i in range(len(X_test)):
    num = np.argmax(probabilities[i])
    if(num != y_test[i]):
        print('Expected emoji:'+ label_to_emoji(y_test[i]) + ' prediction: '+ X_test[i] + label_to_emoji(num).strip())

Expected emoji:😄 prediction: he got a very nice raise😞
Expected emoji:😄 prediction: she got me a nice present❤️
Expected emoji:😞 prediction: This girl is messing with me❤️
Expected emoji:😄 prediction: Congratulation for having a baby❤️
Expected emoji:😄 prediction: you brighten my day❤️
Expected emoji:🍴 prediction: I boiled rice😞
Expected emoji:😞 prediction: she is a bully❤️
Expected emoji:❤️ prediction: My grandmother is the love of my life😄
Expected emoji:😄 prediction: will you be my valentine❤️
Expected emoji:⚾️ prediction: he can pitch really well😞
Expected emoji:😄 prediction: I like to laugh❤️
Expected emoji:😄 prediction: What you did was awesome😞
Expected emoji:😞 prediction: go away⚾️
Expected emoji:😞 prediction: yesterday we lost again⚾️
Expected emoji:❤️ prediction: family is all I have😞
Expected emoji:😄 prediction: You deserve this nice prize😞
Expected emoji:🍴 prediction: I did not have breakfast 😞
